In [7]:
import pandas as pd


In [8]:
df = pd.read_csv('/Users/paolaantonicoli/Desktop/ADM/2020-Jan.csv', header = 'infer', nrows = 200000,parse_dates=['event_time'],
date_parser=pd.to_datetime)

In [9]:
pd.options.display.min_rows = 999


 ## RQ1
A marketing funnel describes your customer’s journey with your e-commerce. 
It may involve different stages, beginning when someone learns about your business, 
when he/she visits your website for the first time, to the purchasing stage, marketing 
funnels map routes to conversion and beyond. Suppose your funnel involves just three simple steps:
1) view, 2) cart, 3) purchase. Which is the rate of complete funnels?


> **What’s the operation users repeat more on average within a session? 
Produce a plot that shows the average number of times users perform each operation 
(view/removefromchart etc etc).**

In [4]:
df.groupby(['event_type']).event_type.count()/(len(df['user_session'].unique()))

event_type
cart                1.193743
purchase            0.196623
remove_from_cart    0.883963
view                2.157117
Name: event_type, dtype: float64

> **How many times, on average, a user views a product before adding it to the cart?**


In [5]:
#? Quando un prodotto è inserito una seconda volta nel carrello, dobbiamo riazzerare il conto delle view???
inseriti_nel_carrello = df.loc[df['event_type']=='cart'][['event_time','product_id','user_id']]
inseriti_nel_carrello.rename(columns={'event_time': 'cart_time'}, inplace=True)

visti = df.loc[df['event_type']=='view'][['event_time','product_id','user_id']]
visti.rename(columns={'event_time': 'view_time'},inplace=True)

prod = pd.merge(visti, inseriti_nel_carrello, how='inner', on=['product_id', 'user_id'])
prod.drop_duplicates()

prod.loc[prod['view_time']<prod['cart_time']].groupby('user_id').count().view_time.mean()

#### VERIFICA IL MERGE SE FUNZIONA
#### PRODOTTI POSSONO ESSERE INSERITI NEL CARRELLO SENZA ESSERE VISTI

4.139858156028369

>**What’s the probability that products added once to the cart are effectively bought?**


In [10]:
#???

> **What’s the average time an item stays in the cart before being removed**



> **How much time passes on average between the first view time and a purchase/addition to cart?**


In [12]:
inseriti = df.loc[df['event_type']=='cart'][['event_time','product_id','user_id','event_type']]
inseriti.rename(columns={'event_time': 'cart_time'}, inplace=True)




inseriti_acquistati = df.loc[(df['event_type']=='cart')|(df['event_type'] =='purchase')][['event_time','product_id','user_id','event_type']]
inseriti_acquistati.rename(columns={'event_time': 'cart_time'}, inplace=True)


visti = df.loc[df['event_type']=='view'][['event_time','product_id','user_id']]
visti.rename(columns={'event_time': 'view_time'},inplace=True)


#visti_per_primi = visti.groupby(['product_id','user_id'],as_index = False).view_time.min()
inseriti_per_primi = inseriti.groupby(['product_id','user_id'])  #.cart_time.min()


for elem in inseriti_per_primi:
    elem.describe()    
    
##Assumendo che per 'inserimento nel carrello' si intenda il primo inserimento del prodotto di un certo tipo nel carrello al momento del primo acquisto
#prod15 = pd.merge(inseriti, visti, how='inner', on=['product_id', 'user_id'])
#prod15.sort_values(['user_id','product_id','cart_time','view_time'])
#p15 = prod15.loc[prod15['view_time']<prod15['cart_time']]
#p15['delta'] = p15['cart_time'] - p15['view_time'] 
#p15.drop_duplicates()

#p15.groupby(['product_id','user_id','cart_time','view_time']).min().mean()

#per ottenere quello a 57 minuti, togliere l'or tra carrello e acquisto

AttributeError: 'tuple' object has no attribute 'describe'

## RQ2
What are the categories of the most trending products overall? For each month visualize this information through a plot showing the number of sold products per category.

> **Plot the most visited subcategories.**

In [14]:
n_cat = df.loc[df['event_type']=='purchase'].groupby(df['category_id']).product_id.count().reset_index(name="count")
n_cat.rename(columns={'count': 'conti'}, inplace=True)

n_cat.sort_values('conti',ascending= False).head()



###1 plot per ogni mese o un plot per entrambi i mesi?
### definire il concetto di sottocategoria

### prodotti più venduti in entrambi i mesi o ciascuno per ogni mese?

,category_id,conti
18,1487580005595612013,684
25,1487580006317032337,375
8,1487580005092295511,349
213,1602943681873052386,299
89,1487580009286598681,279


> **What are the 10 most sold products per category?**

In [15]:
n_cat = df.loc[df['event_type']=='purchase'].groupby([df['category_id'],df['product_id']]).product_id.count().reset_index(name="count")
n_cat.rename(columns={'count': 'conti'}, inplace=True)

n_catt = n_cat.sort_values('conti',ascending= False)
n_catt 

for category_id, frame in n_catt.groupby('category_id'):
    print(frame.head(10), end="\n\n")

           category_id  product_id  conti
0  1487580004832248652     5733049      1
2  1487580004832248652     5842709      1
1  1487580004832248652     5779233      1

            category_id  product_id  conti
9   1487580004857414477     5692322      2
3   1487580004857414477     5689627      1
4   1487580004857414477     5689712      1
5   1487580004857414477     5690409      1
6   1487580004857414477     5690779      1
7   1487580004857414477     5692280      1
8   1487580004857414477     5692284      1
10  1487580004857414477     5746488      1
11  1487580004857414477     5746495      1
12  1487580004857414477     5775411      1

            category_id  product_id  conti
40  1487580004882580302     5727535      5
41  1487580004882580302     5727536      4
35  1487580004882580302     5699779      3
32  1487580004882580302     5699737      1
34  1487580004882580302     5699775      1
36  1487580004882580302     5699807      1
37  1487580004882580302     5699810      1
38  148758000

3534  1487580012591710363     5895782      1

              category_id  product_id  conti
3542  1487580012642042013     5837127      3
3543  1487580012642042013     5869303      1
3541  1487580012642042013     5743931      1
3540  1487580012642042013     5740777      1
3539  1487580012642042013     5618397      1
3538  1487580012642042013     5540780      1

              category_id  product_id  conti
3549  1487580012683985055     5915867      1
3548  1487580012683985055     5915854      1
3547  1487580012683985055     5915853      1
3546  1487580012683985055     5915852      1
3545  1487580012683985055     5870673      1
3544  1487580012683985055     5759385      1

              category_id  product_id  conti
3550  1487580012717539489       29372      1

              category_id  product_id  conti
3551  1487580012876923048     5708350      2

              category_id  product_id  conti
3557  1487580012902088873     5764716      5
3552  1487580012902088873     5651975      2
3559 

## RQ3
For each category, what’s the brand whose prices are higher on average?

> **Write a function that asks the user a category in input and returns a plot indicating the average price of the products sold by the brand.**


In [1]:
def hap_brand():
    c = input()
    assert c in set(df['category_code'])
    df1 = df.loc[df['category_code']==c][['brand','product_id','price']]
    df1 = df1.drop_duplicates()
    return df1.groupby('brand').price.mean()






> **Find, for each category, the brand with the highest average price. Return all the results in ascending order by price.**


In [1]:
df1 = df.groupby(['category_code','brand']).price.mean().reset_index(name="media")

df2= pd.DataFrame(columns=df1.columns)

for elem, frame in df1.groupby('category_code'):
    result = frame.sort_values('media',ascending=False).head(1)
    df2 = pd.concat([df2,result])




NameError: name 'df' is not defined

## [RQ4] 

How much does each brand earn per month? 

>**Write a function that given the name of a brand in input returns, for each month, its profit. Is the average price of products of different brands significantly different? Using the function you just created, find the top 3 brands that have suffered the biggest losses in earnings between one month and the next, specifing bothe the loss percentage and the 2 months (e.g., brand_1 lost 20% between march and april).**

In [ ]:
def as_brand():
    c = input()
    OctPur = dfOct.loc[(dfOct['event_type']=='purchase')&(dfOct['brand']==c)]
    NovPur = dfNov.loc[(dfNov['event_type']=='purchase')&(dfNov['brand']==c)]
    return OctPur.price.mean(),NovPur.price.mean()

In [ ]:
def a_brand(c):
    OctPur = dfOct.loc[(dfOct['event_type']=='purchase')&(dfOct['brand']==c)]
    NovPur = dfNov.loc[(dfNov['event_type']=='purchase')&(dfNov['brand']==c)]
    return OctPur.price.mean(),NovPur.price.mean()

def percentage_loss(o,n):
    if o == 0:
        return str(0)+'%'
    else:
        return str(((n-o)/o)*100)+'%'

brands ={'brand':[],'earnsOct':[],'earnsNov':[],'loss':[]}
brands = pd.DataFrame()

OctPur = dfOct.loc[dfOct['event_type']=='purchase'].drop_duplicates()
NovPur = dfNov.loc[dfNov['event_type']=='purchase'].drop_duplicates()

brands['brand'] = pd.concat([OctPur['brand'],NovPur['brand']])
bra = brands.drop_duplicates()
bra['earnsOct'] = bra['brand'].map(lambda b: a_brand(b)[0]).fillna(0)
bra['earnsNov'] = bra['brand'].map(lambda b: a_brand(b)[1]).fillna(0)
bra['loss'] = bra.apply(lambda x: percentage_loss(o = x['earnsOct'], n = x['earnsNov']), axis=1)



## [RQ5] 
In what part of the day is your store most visited? 
Knowing which days of the week or even which hours of the day shoppers are likely to visit your online store and make a purchase may help you improve your strategies
> **Create a plot that for each day of the week show the hourly average of visitors your store has.**

In [ ]:
import datetime
df1Nov = dfNov.copy()
df1Oct = dfOct.copy()

df1Nov['week_number_of_year'] = df1Nov['event_time'].dt.week
df1Oct['week_number_of_year'] = df1Oct['event_time'].dt.week

df1Nov['weekday'] = df1Nov['event_time'].apply(lambda x: x.weekday())
df1Oct['weekday'] = df1Oct['event_time'].apply(lambda x: x.weekday())

df1Nov['datehour'] = df1Nov['event_time'].dt.hour
df1Oct['datehour'] = df1Oct['event_time'].dt.hour



df2Nov = df1Nov.groupby(['week_number_of_year','weekday','datehour']).user_id.nunique().to_frame(name= 'conto').reset_index()
df2Oct = df1Oct.groupby(['week_number_of_year','weekday','datehour']).user_id.nunique().to_frame(name= 'conto').reset_index()

df3Nov = df2Nov.groupby(['weekday','datehour']).mean()
df3Oct = df2Oct.groupby(['weekday','datehour']).mean()


df3Oct

###df.groupby(['event_type','category_code']).product_id.nunique().groupby('event_type').mean()
###Riguarda gli acquisti o le visite in toto?
### Assumiamo che un utente visiti il sito ad una data ora considerando la prima azione effettuata a quell'ora

The conversion rate of a product is given by the number of times a product has been bought over the number of times it has been visited.
>**What's the conversion rate of your online store?
Find the overall conversion rate of your store.**

In [ ]:
 conversion_rate(v, a):
    if v == 0 :
        return 0 
    else:
        return a/v

prods ={'prods':[],'visitati_Nov':[],'visitati_Oct':[],'acquistati_Nov':[],'acquistati_Oct':[]}
prods = pd.DataFrame()

prods['visitati_Nov'] = dfNov.loc[dfNov['event_type']=='view'].groupby('product_id').event_type.count()
prods['visitati_Oct'] = dfOct.loc[dfNov['event_type']=='view'].groupby('product_id').event_type.count()

prods['acquistati_Nov'] = dfNov.loc[dfNov['event_type']=='purchase'].groupby('product_id').event_type.count()
prods['acquistati_Oct'] = dfOct.loc[dfNov['event_type']=='purchase'].groupby('product_id').event_type.count()

prods['conversion_rate_Oct'] = prods.apply(lambda x: conversion_rate(v = x['visitati_Oct'],a = x['acquistati_Oct']),axis=1)
prods['conversion_rate_Nov'] = prods.apply(lambda x: conversion_rate(v = x['visitati_Nov'],a = x['acquistati_Nov']),axis=1)

prods['mean_conversion_rate'] = prods.apply(lambda x: (x['conversion_rate_Oct'] + x['conversion_rate_Nov'])/2,axis=1)

overall_mean_conversion_rate = prods['mean_conversion_rate'].fillna(0).mean()
prods.sorted_values('mean_conversion_rate', ascending = False)

